# nltkの文章群にscikit-learnを用いてクラスタリングを適用してみる

## 導入編

### 必要なライブラリ・データセットのインポート

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import nltk
import collections

In [ ]:
from sklearn.cluster import k_means_
from sklearn.metrics.pairwise import cosine_similarity, pairwise_distances
from sklearn.preprocessing import StandardScaler


def create_cluster(sparse_data, nclust = 10):

    # Manually override euclidean
    def euc_dist(X, Y = None, Y_norm_squared = None, squared = False):
        #return pairwise_distances(X, Y, metric = 'cosine', n_jobs = 10)
        return cosine_similarity(X, Y)
    k_means_.euclidean_distances = euc_dist
    
    scaler = StandardScaler(with_mean=False)
    sparse_data = scaler.fit_transform(sparse_data)
    kmeans = k_means_.KMeans(n_clusters = nclust, n_jobs = 20, random_state = 3425)
    _ = kmeans.fit(sparse_data)
    return kmeans.labels_

### 今回は以下のnltkの機能を使用できる様にする


In [ ]:
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("reuters")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package reuters to /root/nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### データを取得

In [ ]:
from nltk.corpus import reuters as corpus

### datasetの中身を確認

In [ ]:
for n,item in enumerate(corpus.words(corpus.fileids()[0])[:300]):
    print(item, end=" ")
    if (n%25) ==24:
      print(" ")

ASIAN EXPORTERS FEAR DAMAGE FROM U . S .- JAPAN RIFT Mounting trade friction between the U . S . And Japan has raised fears  
among many of Asia ' s exporting nations that the row could inflict far - reaching economic damage , businessmen and officials said . They  
told Reuter correspondents in Asian capitals a U . S . Move against Japan might boost protectionist sentiment in the U . S . And  
lead to curbs on American imports of their products . But some exporters said that while the conflict would hurt them in the long -  
run , in the short - term Tokyo ' s loss might be their gain . The U . S . Has said it will  
impose 300 mln dlrs of tariffs on imports of Japanese electronics goods on April 17 , in retaliation for Japan ' s alleged failure to  
stick to a pact not to sell semiconductors on world markets at below cost . Unofficial Japanese estimates put the impact of the tariffs at  
10 billion dlrs and spokesmen for major electronics firms said they would virtually halt exports 

### 全document数

In [ ]:
len(corpus.fileids())

10788

### (例) 前からk個のdocumentのみで学習する場合

In [ ]:
k = 100
docs=[corpus.words(fileid) for fileid in corpus.fileids()[:k]]
print(docs)
print("num of docs:", len(docs))

[['ASIAN', 'EXPORTERS', 'FEAR', 'DAMAGE', 'FROM', 'U', ...], ['CHINA', 'DAILY', 'SAYS', 'VERMIN', 'EAT', '7', '-', ...], ['JAPAN', 'TO', 'REVISE', 'LONG', '-', 'TERM', ...], ['THAI', 'TRADE', 'DEFICIT', 'WIDENS', 'IN', 'FIRST', ...], ['INDONESIA', 'SEES', 'CPO', 'PRICE', 'RISING', ...], ['AUSTRALIAN', 'FOREIGN', 'SHIP', 'BAN', 'ENDS', 'BUT', ...], ['INDONESIAN', 'COMMODITY', 'EXCHANGE', 'MAY', ...], ['SRI', 'LANKA', 'GETS', 'USDA', 'APPROVAL', 'FOR', ...], ['WESTERN', 'MINING', 'TO', 'OPEN', 'NEW', 'GOLD', ...], ['SUMITOMO', 'BANK', 'AIMS', 'AT', 'QUICK', 'RECOVERY', ...], ['SUBROTO', 'SAYS', 'INDONESIA', 'SUPPORTS', 'TIN', ...], ['BUNDESBANK', 'ALLOCATES', '6', '.', '1', 'BILLION', ...], ['BOND', 'CORP', 'STILL', 'CONSIDERING', 'ATLAS', ...], ['CHINA', 'INDUSTRIAL', 'OUTPUT', 'RISES', 'IN', ...], ['JAPAN', 'MINISTRY', 'SAYS', 'OPEN', 'FARM', 'TRADE', ...], ['AMATIL', 'PROPOSES', 'TWO', '-', 'FOR', '-', 'FIVE', ...], ['BOWATER', '1986', 'PRETAX', 'PROFITS', 'RISE', '15', ...], ['U', '.

## 前処理編

### 例 : ストップワードリストの作成

### nltkのストップワードリスト

In [ ]:
en_stop = nltk.corpus.stopwords.words('english')

### 例:【発展】記号や数字は正規表現で消してみる

In [ ]:
en_stop= ["``","/",",.",".,",";","--",":",")","(",'"','&',"'",'),',',"','-','.,','.,"','.-',"?",">","<"]                  \
         +["0","1","2","3","4","5","6","7","8","9","10","11","12","86","1986","1987","000"]                                                      \
         +["said","say","u","v","mln","ct","net","dlrs","tonne","pct","shr","nil","company","lt","share","year","billion","price"]          \
         +en_stop

### 前処理関数の作成

In [ ]:
from nltk.corpus import wordnet as wn #lemmatize関数のためのimport

def preprocess_word(word, stopwordset):
    
    #1.make words lower ex: Python =>python
    word=word.lower()
    
    #2.remove "," and "."
    if word in [",","."]:
        return None
    
    #3.remove stopword  ex: the => (None) 
    if word in stopwordset:
        return None
    
    #4.lemmatize  ex: cooked=>cook
    lemma = wn.morphy(word)
    if lemma is None:
        return word

    elif lemma in stopwordset: #lemmatizeしたものがstopwordである可能性がある
        return None
    else:
        return lemma
    

def preprocess_document(document):
    document=[preprocess_word(w, en_stop) for w in document]
    document=[w for w in document if w is not None]
    return document

def preprocess_documents(documents):
    return [preprocess_document(document) for document in documents]

### 前処理の結果を出力してみる

### 前処理前

In [ ]:
print(docs[0][:25]) 

['ASIAN', 'EXPORTERS', 'FEAR', 'DAMAGE', 'FROM', 'U', '.', 'S', '.-', 'JAPAN', 'RIFT', 'Mounting', 'trade', 'friction', 'between', 'the', 'U', '.', 'S', '.', 'And', 'Japan', 'has', 'raised', 'fears']


In [ ]:
print(docs[0][:200])

['ASIAN', 'EXPORTERS', 'FEAR', 'DAMAGE', 'FROM', 'U', '.', 'S', '.-', 'JAPAN', 'RIFT', 'Mounting', 'trade', 'friction', 'between', 'the', 'U', '.', 'S', '.', 'And', 'Japan', 'has', 'raised', 'fears', 'among', 'many', 'of', 'Asia', "'", 's', 'exporting', 'nations', 'that', 'the', 'row', 'could', 'inflict', 'far', '-', 'reaching', 'economic', 'damage', ',', 'businessmen', 'and', 'officials', 'said', '.', 'They', 'told', 'Reuter', 'correspondents', 'in', 'Asian', 'capitals', 'a', 'U', '.', 'S', '.', 'Move', 'against', 'Japan', 'might', 'boost', 'protectionist', 'sentiment', 'in', 'the', 'U', '.', 'S', '.', 'And', 'lead', 'to', 'curbs', 'on', 'American', 'imports', 'of', 'their', 'products', '.', 'But', 'some', 'exporters', 'said', 'that', 'while', 'the', 'conflict', 'would', 'hurt', 'them', 'in', 'the', 'long', '-', 'run', ',', 'in', 'the', 'short', '-', 'term', 'Tokyo', "'", 's', 'loss', 'might', 'be', 'their', 'gain', '.', 'The', 'U', '.', 'S', '.', 'Has', 'said', 'it', 'will', 'impose'

In [ ]:
print(docs[1][:100])

['CHINA', 'DAILY', 'SAYS', 'VERMIN', 'EAT', '7', '-', '12', 'PCT', 'GRAIN', 'STOCKS', 'A', 'survey', 'of', '19', 'provinces', 'and', 'seven', 'cities', 'showed', 'vermin', 'consume', 'between', 'seven', 'and', '12', 'pct', 'of', 'China', "'", 's', 'grain', 'stocks', ',', 'the', 'China', 'Daily', 'said', '.', 'It', 'also', 'said', 'that', 'each', 'year', '1', '.', '575', 'mln', 'tonnes', ',', 'or', '25', 'pct', ',', 'of', 'China', "'", 's', 'fruit', 'output', 'are', 'left', 'to', 'rot', ',', 'and', '2', '.', '1', 'mln', 'tonnes', ',', 'or', 'up', 'to', '30', 'pct', ',', 'of', 'its', 'vegetables', '.', 'The', 'paper', 'blamed', 'the', 'waste', 'on', 'inadequate', 'storage', 'and', 'bad', 'preservation', 'methods', '.', 'It', 'said', 'the', 'government']


### 前処理後

In [ ]:
print(preprocess_documents(docs)[0][:25])

['asian', 'exporter', 'fear', 'damage', 'japan', 'rift', 'mounting', 'trade', 'friction', 'japan', 'raise', 'fear', 'among', 'many', 'asia', 'exporting', 'nation', 'row', 'could', 'inflict', 'far', 'reaching', 'economic', 'damage', 'businessmen']


## クラスタリング編

### tf idfで上記の前処理済みの文章をベクトル化
### vectorizerを使用する（ハイパーパラメーターの設定）

In [ ]:
pre_docs=preprocess_documents(docs)
pre_docs=["".join(doc) for doc in pre_docs]
print(pre_docs[0])

vectorizer = TfidfVectorizer(max_features=200, token_pattern=u'(?u)\\b\\w+\\b',stop_words='english' )

asianexporterfeardamagejapanriftmountingtradefrictionjapanraisefearamongmanyasiaexportingnationrowcouldinflictfarreachingeconomicdamagebusinessmenofficialtellreutercorrespondentasiancapitalmovejapanmightboostprotectionistsentimentleadcurbamericanimportproductexporterconflictwouldhurtlongrunshorttermtokyolossmightgainimpose300tariffimportjapaneseelectronicsgoodapril17retaliationjapanallegefailurestickpactsellsemiconductorworldmarketcostunofficialjapaneseestimateputimpacttariffspokesmanmajorelectronicsfirmwouldvirtuallyhaltexportproducthitnewtaxablebusinessspokesmanleadingjapaneseelectronicsfirmmatsushitaelectricindustrialcoltdmc>.tariffremainplacelengthtimebeyondmonthmeancompleteerosionexportgoodsubjecttarifftommurthastockanalysttokyoofficebrokerjamescapelco>.taiwanbusinessmenofficialalsoworryawareseriousnessthreatjapanservewarningusseniortaiwanesetradeofficialasknametaiwantradetradesurplus15last95surplushelpswelltaiwanforeignexchangereserves53amongworldlargemustquicklyopenmarketremovet

In [ ]:
pre_docs=preprocess_documents(docs)
pre_docs=["".join(doc) for doc in pre_docs]
print(pre_docs[0])

vectorizer = TfidfVectorizer( token_pattern=u'(?u)\\b\\w+\\b')

asianexporterfeardamagejapanriftmountingtradefrictionjapanraisefearamongmanyasiaexportingnationrowcouldinflictfarreachingeconomicdamagebusinessmenofficialtellreutercorrespondentasiancapitalmovejapanmightboostprotectionistsentimentleadcurbamericanimportproductexporterconflictwouldhurtlongrunshorttermtokyolossmightgainimpose300tariffimportjapaneseelectronicsgoodapril17retaliationjapanallegefailurestickpactsellsemiconductorworldmarketcostunofficialjapaneseestimateputimpacttariffspokesmanmajorelectronicsfirmwouldvirtuallyhaltexportproducthitnewtaxablebusinessspokesmanleadingjapaneseelectronicsfirmmatsushitaelectricindustrialcoltdmc>.tariffremainplacelengthtimebeyondmonthmeancompleteerosionexportgoodsubjecttarifftommurthastockanalysttokyoofficebrokerjamescapelco>.taiwanbusinessmenofficialalsoworryawareseriousnessthreatjapanservewarningusseniortaiwanesetradeofficialasknametaiwantradetradesurplus15last95surplushelpswelltaiwanforeignexchangereserves53amongworldlargemustquicklyopenmarketremovet

### fitする

In [ ]:
tf_idf = vectorizer.fit_transform(pre_docs)

In [ ]:
print(tf_idf)

  (0, 108)	0.5
  (0, 119)	0.5
  (0, 120)	0.5
  (0, 5)	0.5
  (1, 23)	1.0
  (2, 71)	1.0
  (3, 122)	1.0
  (4, 62)	1.0
  (5, 8)	1.0
  (6, 97)	0.7071067811865476
  (6, 60)	0.7071067811865476
  (7, 113)	1.0
  (8, 134)	1.0
  (9, 22)	0.7071067811865476
  (9, 117)	0.7071067811865476
  (10, 109)	0.7071067811865476
  (10, 116)	0.7071067811865476
  (11, 17)	1.0
  (12, 12)	0.7071067811865476
  (12, 14)	0.7071067811865476
  (13, 24)	1.0
  (14, 70)	1.0
  (15, 1)	1.0
  (16, 16)	1.0
  (17, 78)	1.0
  :	:
  (78, 101)	1.0
  (79, 55)	1.0
  (80, 6)	0.7071067811865476
  (80, 10)	0.7071067811865476
  (81, 44)	1.0
  (82, 126)	1.0
  (83, 114)	1.0
  (84, 82)	0.7071067811865476
  (84, 77)	0.7071067811865476
  (85, 86)	1.0
  (86, 2)	1.0
  (87, 0)	1.0
  (88, 124)	1.0
  (89, 96)	1.0
  (90, 84)	1.0
  (91, 20)	1.0
  (92, 41)	1.0
  (93, 90)	1.0
  (94, 47)	1.0
  (95, 48)	1.0
  (96, 34)	1.0
  (97, 75)	1.0
  (98, 35)	1.0
  (99, 133)	0.7071067811865476
  (99, 132)	0.7071067811865476


### K-means
### kmeansの設定

In [ ]:
num_clusters = 8
km = KMeans(n_clusters=num_clusters, random_state = 0)

### fitする

In [ ]:
clusters = km.fit_predict(tf_idf)

### 出力結果

In [ ]:
for doc, cls in zip(pre_docs, clusters):
    print(cls,doc)

0 asianexporterfeardamagejapanriftmountingtradefrictionjapanraisefearamongmanyasiaexportingnationrowcouldinflictfarreachingeconomicdamagebusinessmenofficialtellreutercorrespondentasiancapitalmovejapanmightboostprotectionistsentimentleadcurbamericanimportproductexporterconflictwouldhurtlongrunshorttermtokyolossmightgainimpose300tariffimportjapaneseelectronicsgoodapril17retaliationjapanallegefailurestickpactsellsemiconductorworldmarketcostunofficialjapaneseestimateputimpacttariffspokesmanmajorelectronicsfirmwouldvirtuallyhaltexportproducthitnewtaxablebusinessspokesmanleadingjapaneseelectronicsfirmmatsushitaelectricindustrialcoltdmc>.tariffremainplacelengthtimebeyondmonthmeancompleteerosionexportgoodsubjecttarifftommurthastockanalysttokyoofficebrokerjamescapelco>.taiwanbusinessmenofficialalsoworryawareseriousnessthreatjapanservewarningusseniortaiwanesetradeofficialasknametaiwantradetradesurplus15last95surplushelpswelltaiwanforeignexchangereserves53amongworldlargemustquicklyopenmarketremov

In [ ]:
num_clusters = 5
km = KMeans(n_clusters=num_clusters, random_state = 0)

In [ ]:
clusters = km.fit_predict(tf_idf)

In [ ]:
for doc, cls in zip(pre_docs, clusters):
    print(cls,doc)

0 asianexporterfeardamagejapanriftmountingtradefrictionjapanraisefearamongmanyasiaexportingnationrowcouldinflictfarreachingeconomicdamagebusinessmenofficialtellreutercorrespondentasiancapitalmovejapanmightboostprotectionistsentimentleadcurbamericanimportproductexporterconflictwouldhurtlongrunshorttermtokyolossmightgainimpose300tariffimportjapaneseelectronicsgoodapril17retaliationjapanallegefailurestickpactsellsemiconductorworldmarketcostunofficialjapaneseestimateputimpacttariffspokesmanmajorelectronicsfirmwouldvirtuallyhaltexportproducthitnewtaxablebusinessspokesmanleadingjapaneseelectronicsfirmmatsushitaelectricindustrialcoltdmc>.tariffremainplacelengthtimebeyondmonthmeancompleteerosionexportgoodsubjecttarifftommurthastockanalysttokyoofficebrokerjamescapelco>.taiwanbusinessmenofficialalsoworryawareseriousnessthreatjapanservewarningusseniortaiwanesetradeofficialasknametaiwantradetradesurplus15last95surplushelpswelltaiwanforeignexchangereserves53amongworldlargemustquicklyopenmarketremov

In [ ]:
num_clusters = 3
km = KMeans(n_clusters=num_clusters, random_state = 0)

In [ ]:
clusters = km.fit_predict(tf_idf)

In [ ]:
for doc, cls in zip(pre_docs, clusters):
    print(cls,doc)

0 asianexporterfeardamagejapanriftmountingtradefrictionjapanraisefearamongmanyasiaexportingnationrowcouldinflictfarreachingeconomicdamagebusinessmenofficialtellreutercorrespondentasiancapitalmovejapanmightboostprotectionistsentimentleadcurbamericanimportproductexporterconflictwouldhurtlongrunshorttermtokyolossmightgainimpose300tariffimportjapaneseelectronicsgoodapril17retaliationjapanallegefailurestickpactsellsemiconductorworldmarketcostunofficialjapaneseestimateputimpacttariffspokesmanmajorelectronicsfirmwouldvirtuallyhaltexportproducthitnewtaxablebusinessspokesmanleadingjapaneseelectronicsfirmmatsushitaelectricindustrialcoltdmc>.tariffremainplacelengthtimebeyondmonthmeancompleteerosionexportgoodsubjecttarifftommurthastockanalysttokyoofficebrokerjamescapelco>.taiwanbusinessmenofficialalsoworryawareseriousnessthreatjapanservewarningusseniortaiwanesetradeofficialasknametaiwantradetradesurplus15last95surplushelpswelltaiwanforeignexchangereserves53amongworldlargemustquicklyopenmarketremov

In [ ]:
num_clusters = 10
km = KMeans(n_clusters=num_clusters, random_state = 0)

In [ ]:
clusters = km.fit_predict(tf_idf)

In [ ]:
for doc, cls in zip(pre_docs, clusters):
    print(cls,doc)

0 asianexporterfeardamagejapanriftmountingtradefrictionjapanraisefearamongmanyasiaexportingnationrowcouldinflictfarreachingeconomicdamagebusinessmenofficialtellreutercorrespondentasiancapitalmovejapanmightboostprotectionistsentimentleadcurbamericanimportproductexporterconflictwouldhurtlongrunshorttermtokyolossmightgainimpose300tariffimportjapaneseelectronicsgoodapril17retaliationjapanallegefailurestickpactsellsemiconductorworldmarketcostunofficialjapaneseestimateputimpacttariffspokesmanmajorelectronicsfirmwouldvirtuallyhaltexportproducthitnewtaxablebusinessspokesmanleadingjapaneseelectronicsfirmmatsushitaelectricindustrialcoltdmc>.tariffremainplacelengthtimebeyondmonthmeancompleteerosionexportgoodsubjecttarifftommurthastockanalysttokyoofficebrokerjamescapelco>.taiwanbusinessmenofficialalsoworryawareseriousnessthreatjapanservewarningusseniortaiwanesetradeofficialasknametaiwantradetradesurplus15last95surplushelpswelltaiwanforeignexchangereserves53amongworldlargemustquicklyopenmarketremov

## ヒント

<p1>
scikit-learnのvectorizerとkmeansにはたくさんのハイパーパラメータがあります。vectorizerのハイパーパラメータの中には前処理機能(例：stop_words)もあります。
    ハイパーパラメータの設定を変える事で最終的な結果は変わります。以下のURLにアクセスしてハイパーパラメータの独自で設定してみてください。<br>
    ・TF-IDFに関するパラメータ<br>
    https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html<br>
    ・Kmeansに関するパラメータ<br>
    https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html<br>
</p1>


## 応用
<p1>
    クラスタリング編でコードを以下に指示に従って変更する事で結果がどの様に変わるのかを確認してみましょう<br>
    （１）講義で学んだ他の手法でベクトル化してみる(例：bag-of-words)<br>
    （２）kmeans以外の手法、又はkmeansを可視化してみる(例：階層型クラスタリング)<br>
<p1>